In [13]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from ridge import Ridge_
from sklearn.metrics import mean_absolute_error

In [2]:
df = pd.read_csv('olympics_100m.csv', sep=',')
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [3]:
predictor = 'Year'
target = 'Time'

In [4]:
X = train[predictor].values
y = train[target].values

In [5]:
X_test = test[predictor].values
y_test = test[target].values

In [6]:
X_test.shape
len(train)

23

In [7]:
# don't know if required ??
# Centering and normalizing the data so that the effect of the intercept
# (bias) is not so relevant
x_mean = X.mean()
x_std = X.std()

# scale the data
X = (X - x_mean) / x_std
X = X.reshape(len(train), 1)

In [8]:
X_test = (X_test - x_mean) / x_std
X_test = X_test.reshape(len(test), 1)

In [9]:
poly = PolynomialFeatures(1)
X = poly.fit_transform(X)
X_test = poly.fit_transform(X_test)

In [10]:
lambda_val = 2

In [12]:
ridge_model = Ridge_(lambda_val)
w = ridge_model.ridge_regul(X, y)

array([ 9.5068    , -0.46576763])

In [14]:
errors = []
lambdas = [10**i for i in range(-5, 3)]
for l in lambdas:
    w, x_mean, x_std = ridge_model.fit(train, predictor, target, l)
    preds = ridge_model.predict(test, predictor, x_mean, x_std, w)
    errors.append(mean_absolute_error(test[target], preds))
    

[1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]